In [1]:
import pandas as pd
import numpy as np
import os
import glob
import re
from sklearn.preprocessing import StandardScaler

# Create output directory
output_dir = r"C:\Users\Ivan Nathanael\Uni\Side Project\CSI_Location_Tracking\csi-project\data_preprocessing"
os.makedirs(output_dir, exist_ok=True)

def preprocess_csi_data(data_dir):
    """
    Preprocess CSI data from multiple CSV files across different environments
    for walking vs. standing classification.
    
    Parameters:
    data_dir: Directory containing CSV files with naming convention indicating activity
              (should contain 'walking' or 'standing' in filename)
    """
    # Get all CSV files in the directory and subdirectories
    csv_files = []
    for root, _, files in os.walk(data_dir):
        for file in files:
            if file.lower().endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    
    if not csv_files:
        raise ValueError(f"No CSV files found in {data_dir}")
    
    print(f"Found {len(csv_files)} CSV files")
    
    # Dictionary to group files by activity
    activity_files = {
        'walking': [],
        'standing': []
    }
    
    # Categorize files by activity
    for file_path in csv_files:
        filename = os.path.basename(file_path).lower()
        
        if 'walking' in filename:
            activity_files['walking'].append(file_path)
        elif 'standing' in filename:
            activity_files['standing'].append(file_path)
        else:
            print(f"Skipping file {filename} - activity not identified in filename")
    
    print(f"Found {len(activity_files['walking'])} walking files and {len(activity_files['standing'])} standing files")
    
    # Function to extract CSI values from the 'data' column
    def extract_csi_values(data_str):
        try:
            if isinstance(data_str, str):
                # Remove square brackets and split by commas
                values_str = re.findall(r'-?\d+\.?\d*', data_str)
                # Convert to float
                values = [float(val.strip()) for val in values_str]
                return values
            return None
        except (ValueError, AttributeError) as e:
            print(f"Error processing CSI data: {e}")
            return None
    
    # Lists to store all data
    all_csi_data = []
    all_labels = []
    environment_info = []  # To track which environment each sample came from
    
    # Process each activity
    for activity, files in activity_files.items():
        activity_label = 1 if activity == 'walking' else 0
        
        print(f"\nProcessing {activity} activity files...")
        
        # Process each file in the activity
        for env_idx, file_path in enumerate(files):
            filename = os.path.basename(file_path)
            env_name = os.path.dirname(file_path).split(os.sep)[-1]  # Get environment name from directory
            
            print(f"  Processing file {filename} from environment: {env_name}")
            
            try:
                # Read CSV file
                df = pd.read_csv(file_path, quotechar='"')
                
                # Extract CSI data
                csi_values = df['data'].apply(extract_csi_values).dropna().tolist()
                
                if not csi_values:
                    print(f"  No valid CSI data found in {filename}, skipping")
                    continue
                    
                print(f"  Extracted {len(csi_values)} valid CSI samples")
                
                # Add data, labels, and environment info to our lists
                all_csi_data.extend(csi_values)
                all_labels.extend([activity_label] * len(csi_values))
                environment_info.extend([f"{activity}_{env_name}_{env_idx}"] * len(csi_values))
                
            except Exception as e:
                print(f"  Error processing {filename}: {e}")
    
    # Check if we have enough data
    if not all_csi_data:
        raise ValueError("No valid CSI data was extracted from any files")
    
    print(f"\nTotal extracted samples: {len(all_csi_data)}")
    
        # Determine the most common length of CSI vectors
    length_counts = {}
    for csi in all_csi_data:
        length = len(csi)
        length_counts[length] = length_counts.get(length, 0) + 1
    
    # Sort by count to find the most common length
    sorted_lengths = sorted(length_counts.items(), key=lambda x: x[1], reverse=True)
    most_common_length = sorted_lengths[0][0]
    
    print(f"Most common CSI vector length: {most_common_length} (found in {length_counts[most_common_length]} samples)")
    print(f"Length distribution: {sorted_lengths[:5]}")
    
    # Standardize CSI vectors to the same length
    standardized_csi = []
    standardized_labels = []
    kept_environments = []
    
    for csi, label, env in zip(all_csi_data, all_labels, environment_info):
        # Skip samples that are significantly different in length
        if len(csi) < most_common_length * 0.8 or len(csi) > most_common_length * 1.2:
            continue
            
        if len(csi) < most_common_length:
            # Pad with zeros if shorter
            padded_csi = csi + [0] * (most_common_length - len(csi))
            standardized_csi.append(padded_csi)
        else:
            # Truncate if longer
            standardized_csi.append(csi[:most_common_length])
        
        standardized_labels.append(label)
        kept_environments.append(env)
    
    # Convert to numpy arrays
    X = np.array(standardized_csi)
    y = np.array(standardized_labels).reshape(-1, 1)
    
    print(f"\nData shape after standardization: {X.shape}")
    print(f"Label distribution: Walking: {np.sum(y)}, Standing: {len(y) - np.sum(y)}")
    
    # Save environment information for analysis
    env_df = pd.DataFrame({
        'environment': kept_environments,
        'label': standardized_labels
    })
    env_df.to_csv(os.path.join(output_dir, "environment_info.csv"), index=False)
    print(f"Environment information saved to {output_dir}environment_info.csv")
    
    # Check for class imbalance
    walking_count = np.sum(y)
    standing_count = len(y) - walking_count
    print(f"\nClass distribution: Walking: {walking_count} ({walking_count/len(y)*100:.1f}%), "
          f"Standing: {standing_count} ({standing_count/len(y)*100:.1f}%)")
    
    # Normalize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Save the processed data
    np.savez(os.path.join(output_dir, "optimized_features.npz"), 
             data=X_scaled, 
             labels=y)
    
    # Also save the scaler for future use
    import joblib
    joblib.dump(scaler, os.path.join(output_dir, "csi_scaler.joblib"))
    
    print(f"\nPreprocessed data saved to {output_dir}optimized_features.npz")
    print(f"Scaler saved to {output_dir}csi_scaler.joblib")
    
    # Optional: Save a sample of the data for inspection
    sample_idx = np.random.choice(range(len(X_scaled)), min(5, len(X_scaled)), replace=False)
    sample_df = pd.DataFrame(X_scaled[sample_idx])
    sample_df['label'] = y[sample_idx]
    sample_df['environment'] = [kept_environments[i] for i in sample_idx]
    sample_df.to_csv(os.path.join(output_dir, "sample_data.csv"), index=False)
    
    return X_scaled, y

# Directory containing your CSV files
data_dir = r"C:\Users\Ivan Nathanael\Uni\Side Project\CSI_Location_Tracking\csi-project"  # Update this path

# Run preprocessing
X, y = preprocess_csi_data(data_dir)

print("\nPreprocessing complete!")
print(f"Features shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print("\nYou can now run your transformer model using this data.")

Found 9 CSV files
Skipping file csi_data.csv - activity not identified in filename
Skipping file csi_data-checkpoint.csv - activity not identified in filename
Found 4 walking files and 3 standing files

Processing walking activity files...
  Processing file walking_ly8room_csidata.csv from environment: LY8-Room
  Extracted 1932 valid CSI samples
  Processing file walking_ly8room_csidata-checkpoint.csv from environment: .ipynb_checkpoints
  Extracted 1932 valid CSI samples
  Processing file walking_pantry_csidata.csv from environment: LY8_Pantry
  Extracted 1709 valid CSI samples
  Processing file walking_MAKERSPACE_csidata.csv from environment: Makerspace
  Extracted 1896 valid CSI samples

Processing standing activity files...
  Processing file standing_ly8room_csidata.csv from environment: LY8-Room
  Extracted 1730 valid CSI samples
  Processing file standing_pantry_csidata.csv from environment: LY8_Pantry
  Extracted 1340 valid CSI samples
  Processing file standing_MAKERSPACE_csida

In [5]:
import glob
from pathlib import Path
import os

def find_csv_files_recursively(root_dir):
    """
    Recursively searches for CSV files within a directory and its subdirectories.

    Args:
        root_dir (str): The path to the root directory to start the search from.

    Returns:
        list: A list of full paths to all CSV files found.
              Returns an empty list if no CSV files are found or if the root directory is not valid.
    """
    csv_file_paths = []
    if not os.path.isdir(root_dir):
        return csv_file_paths  # Return empty list if root_dir is not a directory

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.lower().endswith(".csv"):  # Case-insensitive check for .csv extension
                csv_file_path = os.path.join(dirpath, filename)
                csv_file_paths.append(csv_file_path)
    return csv_file_paths


folder_path = Path("CSI_raw")
find_csv_files_recursively(folder_path)

[]

In [20]:
import pandas as pd

pd.read_csv("standing_pantry_csidata.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'standing_pantry_csidata.csv'